In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

from modules.unet import UNet_LS, UNet_LS_down, UNet_LS_up
from transfers import UNetTransfer, Transfer, pretrained_transfers
from logger import Logger, VisdomLogger
from datasets import load_train_val, load_test, load_ood
from task_configs import tasks, RealityTask
from model_configs import model_types
from models import DataParallelModel


In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_CUDA = torch.cuda.is_available()

# MULTITASK APPROACH

In [3]:
MULTITASK_MODELS_DIR = "/scratch/kivva/projects/consistency_LS/multitask/results_mae/models"

dest_tasks = [tasks.normal, tasks.depth_zbuffer]
src_task = tasks.rgb

model_rgb_down = model_types["rgb"]["down"][0]()
model_normal_up = model_types["normal"]["up"][0]()
model_depth_up = model_types["depth_zbuffer"]["up"][0]()

In [4]:
path_rgb_down = MULTITASK_MODELS_DIR + '/rgb_down.pth'
path_normal_up = MULTITASK_MODELS_DIR + "/normal_up.pth"
path_depth_up = MULTITASK_MODELS_DIR + "/depth_zbuffer_up.pth"

model_rgb_down.load_weights(path_rgb_down)
model_normal_up.load_weights(path_normal_up)
model_depth_up.load_weights(path_depth_up)

In [5]:
rgb2normal = UNet_LS(model_up=model_normal_up, model_down=model_rgb_down)
rgb2depth = UNet_LS(model_up=model_depth_up, model_down=model_rgb_down)

In [6]:
ood_set = load_ood([tasks.rgb])
ood = RealityTask.from_static("ood",  ood_set, [tasks.rgb])

number of ood images:  21


In [7]:
logger = VisdomLogger("visualize", env="visualize_models")

Setting up a new session...


No deletion
In (git) scaling-reset
Logging to environment visualize_models


In [8]:
input_data = ood.task_data[tasks.rgb].to(DEVICE)

In [9]:
rgb2normal.to(DEVICE)
rgb2depth.to(DEVICE)
rgb2normal = nn.DataParallel(rgb2normal) if not isinstance(rgb2normal, nn.DataParallel) else rgb2normal
rgb2depth = nn.DataParallel(rgb2depth)if not isinstance(rgb2depth, nn.DataParallel) else rgb2depth

In [10]:
with torch.no_grad():
    input_data = input_data.to(DEVICE)
    out_normal = rgb2normal(input_data)
    input_data = input_data.to(DEVICE)
    out_depth = rgb2depth(input_data)
    
    shape = list(out_normal.shape)
    shape[1] = 3
    out_normal = out_normal.clamp(min=0, max=1).expand(*shape)
    out_depth = out_depth.clamp(min=0, max=1).expand(*shape)
    

In [11]:
images = [input_data, out_normal, out_depth]

logger.images_grouped(images, f"multitask_ood:rgb->normal, rgb->depth", resize=256)

# BASELINE APPROACH

In [15]:
BASELINE_MODELS_DIR = "./baseline/models"

dest_tasks = [tasks.normal, tasks.depth_zbuffer]
src_task = tasks.rgb

baseline_rgb2normal, path_rgb2normal = pretrained_transfers[(src_task.name, dest_tasks[0].name)]
baseline_rgb2depth, path_rgb2depth = pretrained_transfers[(src_task.name, dest_tasks[1].name)]
path_rgb2normal = BASELINE_MODELS_DIR + "/unet_baseline_standardval.pth"
path_rgb2depth = BASELINE_MODELS_DIR + "/rgb2zdepth_buffer.pth"
baseline_rgb2depth = baseline_rgb2depth()
baseline_rgb2normal = baseline_rgb2normal()
baseline_rgb2depth = DataParallelModel(baseline_rgb2depth)
baseline_rgb2normal = DataParallelModel(baseline_rgb2normal)

In [15]:
baseline_rgb2depth.load_weights(path_rgb2depth)
baseline_rgb2normal.load_weights(path_rgb2normal)

In [16]:
baseline_rgb2depth.to(DEVICE)
baseline_rgb2normal.to(DEVICE)
input_data = ood.task_data[tasks.rgb].to(DEVICE)

In [17]:
with torch.no_grad():
    out_normal = baseline_rgb2normal(input_data)
    input_data.to(DEVICE)
    out_depth = baseline_rgb2depth(input_data)
    
    shape = list(out_normal.shape)
    shape[1] = 3
    out_normal = out_normal.clamp(min=0, max=1).expand(*shape)
    out_depth = out_depth.clamp(min=0, max=1).expand(*shape)

In [18]:
images = [input_data, out_normal, out_depth]

logger.images_grouped(images, f"baseline_ood:rgb->normal, rgb->depth", resize=256)

# VISUALIZE ANOTHER MODELS

In [12]:
MODELS_DIR = "/home/kivva/Consistency_LD/multitask/results/results_Multitask:rgb->noraml|depth_zbuffer/models"

dest_tasks = [tasks.normal, tasks.depth_zbuffer]
src_task = tasks.rgb

model1_rgb_down = model_types["rgb"]["down"][0]()
model1_normal_up = model_types["normal"]["up"][0]()
model1_depth_up = model_types["depth_zbuffer"]["up"][0]()

In [13]:
path_rgb_down = MULTITASK_MODELS_DIR + '/rgb_down.pth'
path_normal_up = MULTITASK_MODELS_DIR + "/normal_up.pth"
path_depth_up = MULTITASK_MODELS_DIR + "/depth_zbuffer_up.pth"

model1_rgb_down.load_weights(path_rgb_down)
model1_normal_up.load_weights(path_normal_up)
model1_depth_up.load_weights(path_depth_up)

In [14]:
rgb2normal = UNet_LS(model_up=model1_normal_up, model_down=model1_rgb_down)
rgb2depth = UNet_LS(model_up=model1_depth_up, model_down=model1_rgb_down)

In [15]:
input_data = ood.task_data[tasks.rgb].to(DEVICE)

In [16]:
rgb2normal.to(DEVICE)
rgb2depth.to(DEVICE)
rgb2normal = nn.DataParallel(rgb2normal) if not isinstance(rgb2normal, nn.DataParallel) else rgb2normal
rgb2depth = nn.DataParallel(rgb2depth)if not isinstance(rgb2depth, nn.DataParallel) else rgb2depth

In [17]:
with torch.no_grad():
    input_data = input_data.to(DEVICE)
    out_normal = rgb2normal(input_data)
    input_data = input_data.to(DEVICE)
    out_depth = rgb2depth(input_data)
    
    shape = list(out_normal.shape)
    shape[1] = 3
    out_normal = out_normal.clamp(min=0, max=1).expand(*shape)
    out_depth = out_depth.clamp(min=0, max=1).expand(*shape)

In [18]:
images = [input_data, out_normal, out_depth]

logger.images_grouped(images, f"model_ood:rgb->normal, rgb->depth", resize=256)